In [ ]:
# import all needed functions

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, roc_curve, auc
import numpy as np
from sklearn.preprocessing import StandardScaler

In [4]:
# I use Logistic Regression class
# Logistic Regression method from scratch below

class Logistic_Regression:

    def __init__(self, learning_rate=0.001, iterations=1000):
        self.lr = learning_rate
        self.iter = iterations
        self.weights = None
        self.bias = None

    def sigmoid(self, z):
        z_clipped = np.clip(z, -250, 250)
        return 1/ (1 + np.exp(-z_clipped))

    def fit(self, features, target):
        n_samples, n_features = features.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        for _ in range(self.iter):

            linear_model = np.dot(features, self.weights) + self.bias
            predicted = self.sigmoid(linear_model)
            dw = (1 / n_samples) * np.dot(features.T, predicted - target)
            db = (1/ n_samples) * np.sum(predicted - target)

            self.weights -= self.lr * dw
            self.bias -= self.lr * db

    def predict_probability(self, features):
        if self.weights is None:
            raise Exception("Please train your model before predicting probability :)")
        linear_model = np.dot(features, self.weights) + self.bias
        return self.sigmoid(linear_model)

    def final_predict(self, features):
        probabilities = self.predict_probability(features)
        return [1 if i > 0.5 else 0 for i in probabilities]

In [ ]:
# file uploading 

file = input("Please enter here location of your file (without quotation marks!): \n")

In [ ]:
# To read the path of our file correctly, I have created a function that replaces \ with /

def slash_validator(your_file):
    file_path = ""
    to_swap = "\\"
    correct = "/"

    for i in your_file:
        if i == to_swap:
            file_path += correct
        else:
            file_path += i
    print(file_path, "<- correct slashes")
    return file_path

slash_validator(file)

try:
    df = pd.read_csv(file, sep=",", decimal = ',')
    print("file loaded")
except FileNotFoundError:
    print("check file location")
    exit()

In [ ]:
# Let's prepare our data.

# I changed the column names for better readability - instead of spaces I used "_"
# and I also removed spaces at the beginning and at the end of the columns names.

df.columns = [col.strip().replace(" ", "_") for col in df.columns]
print(df.columns)

# Setting target and features which on we will train our model

target_variable = "Cardiomegaly"
features = df.drop(columns=['ID', target_variable])
target = df[target_variable]

features_to_numpy = features.to_numpy()
target_to_numpy = target.to_numpy()

print("Data prepared")

In [ ]:
# Creating lists, we will use them in classification report and to create plots

fold_scores = []
all_true_labels = []
all_predictions = []
all_predicted_probabilities = []

In [ ]:
# We have not enough data in our file (37) so I used cross validation method.
# It uses every fold that we sets training test and then it calculates average score
# of model being right about diseased or healthy heart.

splits = 5
skf = StratifiedKFold(n_splits = splits, shuffle = True, random_state=42)

for fold, (train_index, val_index) in enumerate(skf.split(features_to_numpy, target_to_numpy)):

    features_train, features_val = features_to_numpy[train_index], features_to_numpy[val_index]
    target_train, target_val = target_to_numpy[train_index], target_to_numpy[val_index]

    scaler = StandardScaler()
    features_train_scaled = scaler.fit_transform(features_train)
    features_val_scaled = scaler.transform(features_val)

    model = Logistic_Regression(learning_rate = 0.001, iterations = 1000)
    model.fit(features_train_scaled, target_train)

    target_final_predict = model.final_predict(features_val_scaled)
    target_predicted_probabilities = model.predict_probability(features_val_scaled)

    score = accuracy_score(target_val, target_final_predict)
    fold_scores.append(score)
    all_true_labels.extend(target_val)
    all_predictions.extend(target_final_predict)
    all_predicted_probabilities.extend(target_predicted_probabilities)

In [ ]:
# Getting report about our model (precision- how accurate our model was in detecting a healthy and diseased heart,
#recall- how much % of diseased hearts it detected, 
#f1-score- the harmonic mean, combining both of these measures into one result. Shows how strong a model is).

target_names = ['Healthy heart (0)', 'Cardiomegaly (1)']

print(classification_report(all_true_labels, all_predictions, target_names = target_names))

In [ ]:
# Visualizing data into ROC curve.

plt.figure(figsize=(8,8))
false_pr, true_pr, _ = roc_curve(all_true_labels, all_predicted_probabilities)
roc_auc = auc(false_pr, true_pr)
plt.subplot(1,1,1)
plt.plot(false_pr,true_pr,color='red', lw=2, label = f"roc curve (AUC = {roc_auc})")
plt.plot([0, 1], [0, 1], color='blue', lw=2, linestyle='--', label='random guessing')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve (cross validation)')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()